Install OpenAI pakcages

In [ ]:
!pip install --upgrade openai

Helper function to transform lottie json file into required format 

In [3]:
import json
import os

def create_json_object(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        title = file_path.split("/")[-1].split(".")[0].replace('_', ' ')
        json_obj = {"prompt": "Generate lottie json file based on description: " + title + "\n\n###\n\n", "completion": json.dumps(data) + " END"}
        return json_obj


def create_jsonl(directory_path, output_file_path):
    with open(output_file_path, 'w') as f:
        for file_name in os.listdir(directory_path):
            if file_name.endswith('.json'):
                file_path = os.path.join(directory_path, file_name)
                json_obj = create_json_object(file_path)
                json.dump(json_obj, f)
                f.write('\n')

Loop through json file under folder augmented_files, create output dataset for fine-tuning openAI's model, the result will be saved as out.jsonl

In [4]:
create_jsonl('augmented_files', 'aug.json')

Upload the prepared dataset

In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")
file = openai.File.create(
  file=open("out.jsonl", "rb"),
  purpose='fine-tune'
)
print(file.id)

Start Fine-tune job

In [ ]:

openai.FineTune.list()
ft_obj = openai.FineTune.create(training_file=file.id, model="davinci")
print(ft_obj)

Wait for fine-tune job to be finished. When finished, the model name will be listed: e.g. davinci:ft-personal-2023-08-15-02-12-49


In [ ]:
openai.FineTune.retrieve(id=ft_obj.id).status
ft_model = openai.FineTune.retrieve(id=ft_obj.id).fine_tuned_model
print(ft_model)

Use the newly trained model

In [ ]:
def apply_ft(description, model_name):
    prompt = f"Generate lottie json file based on description: {description}"
    print(prompt)
    result = openai.Completion.create(model=model_name, prompt=prompt, temperature=0, top_p=1, n=1, logprobs=2)
    print("result: " + result['choices'][0]['text'])

apply_ft('white line on black background draw itself from left to right', ft_model)